# 나만의 작은 비서 (하브루타 편)

1. 목소리를 받아 텍스트로 변환
2. 받은 텍스트를 바탕으로 프로젝트 엔지니어링 진행
3. 엔지니어링을 바탕으로 받은 내용을 음성으로 변환
4. "종료"라고 말하기전까지 1~3 반복
5. 대화한 테스트를 리스트의 형식으로 저장

In [1]:
from dotenv import load_dotenv
from openai import OpenAI

import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# 음성 인식 코드

In [ ]:
import speech_recognition as sr

recognizer = sr.Recognizer()

while True:
    with sr.Microphone() as source:
        print('말씀하세요')
        audio = recognizer.listen(source)
        txt = recognizer.recognizer_google(audio, language='ko-KR')
        print(txt)

        if txt == '종료':
            break

# TTS 코드

In [ ]:
client = OpenAI()

text = "아니 그러니깐 BTC 가격을 예측하기 위해서 하향식 구조를 이용해서 긴 시간 단위의 모델을 바탕으로 학습한 내용을 낮은 단계 모델, 즉 상위 모델보다 시간 단위가 짧은 모델에게 가중치를 제공함으로서 시계열 모델이 가격을 정확하게 예측할 수 있도록 만들었는데, 문제는 학습하는 과정에서 모델이 자원을 너무 많이 먹어서 이러한 문제를 해결하기 위해 메모리 뱅킹을 이용해서 특정 구간에서 상승, 하락에 대한 내용을 기억하고 모델이 예측하는 시점에서 메모리 뱅킹에 저장했던 내용을 바탕으로 더 빠르고 자원을 덜 사용할 수 게 개발을 하는데.."

with client.audio.speech.with_streaming_response.create(
    model='tts-1',
    voice='nova',
    input=text
 ) as response:
    response.stream_to_file('tts_output.mp3')

# 프롬프트 엔지니어링 코드

In [ ]:
def correct_title(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    당신은 편집장 입니다 기자들이 숭고한 기사 제목을 교정하세요

    #### 지시사항 ###
    - 기사의 제목이 명확하고 주제와 잘 맞도록 수정할 것
    - 독자의 관심을 끌 수 있도록 간결하고 임팩트 있는 표현을 사용할 것,
    - 비속어, 은어 등은 제거하고 의미가 유지되도록 교정할 것

    ### 출력 형식 ###
    - 원래 제목: [기사의 원래 제목]
    - 교정 제콕:
        [기사의 교정된 제목]
        [기사의 교정된 제목]
        [기사의 교정된 제목]

    ### 예시 ###
    - 원래 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
    - 교정 제목: 
        "서울 대형 화재, 수백명 대피!"
        "서울서 화재, 수백명 대피해..."
        "수뱍명 대피한 서울 화재"    
    """

    user_mmsg = f"""
    다음 제목을 교정해주세요.
    제목 {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system",
                "content": [
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                    
                ]
            },
            {
                "role":"user",
                "content":[
                    {
                        "type":"text",
                        "text": user_mmsg
                    }
                ]
            }
        ],
        response_format={
            "type":"text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1
    )

    return response.choices[0].message.content

In [ ]:
print(correct_title("비 오는 날 개꿀잼 라디오 방송 사연이 나옵니다람쥐"))

# Chat Comlprtion

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "너는 친절하고 상세한 설명을 잘하는 챗봇이야"
        },
        {
            "role" : "user",
            "content": "안녕, 나는 다람쥐 썬더야"
        },
        {
            "role": "assistant",
            "content": " 안녕하세요 람쥐 썬더님, 무엇을 도와드릴까요?"
        },
        {
            "role": "user",
            "content":"내 이름이 뭐라고??"
        }
    ],
    temperature=1,
    max_tokens=4096,
    top_p=1
)

In [ ]:
response.choices[0].message.content

# SST

In [ ]:
with open("gtts_output_2.mp3", "rb") as f:
    transcriptions = client.audio.transcriptions.create(
        model='whisper-1',
        file=f
    )

    print(transcriptions)

# 텍스트 저장 및 출력